In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)
global swiss

In [ ]:
# Import dataset 
data = pd.read_csv('./data/airline.csv')

In [ ]:
def reset():
    global swiss
    swiss = data[data["airline_name"] == "swiss-international-air-lines"]

# Analysiere Kundenbewertungen der Swiss im Zeitraum von 2012 bis 2015

In den Jahren 2012 - 2015 betrug die jährliche Zahl der Fluggäste etwa  17 Mio. (Quelle: https://de.statista.com/statistik/daten/studie/291651/umfrage/fluggaeste-von-swiss/)

In [ ]:
# Datensatz für SWISS Interantional Airlines herausfiltern und Länge ermitteln
swiss = data[data["airline_name"] == "swiss-international-air-lines"]
print(f"Der Datensatz enthält {len(swiss)} Datenpunkte")

# reset DataFrame index
swiss = swiss.reset_index(drop=True)

## Preprocessing

### Data Cleaning

Zuerst werden alle Spalten entfernt, die keinen (unmittelbaren) Einfluss auf die Bewertung des Fluggastes haben (Metadaten):
* **link**: Link zur Datenquelle (irrelevant) -> **drop**
* **author & author_country**: Name und Herkunft des Autors (irrelevant) -> **drop**
* **airline_name**: Für alle Datenpunkte gleich, da nur Swiss betrachtet wird -> **drop**
* **title**: Auch für alle Datenpunkte gleich -> **drop** 
* **date**: Saisonale Effekte und zeitliche Veränderungen sollen vernachlässigt werden -> **drop**

Folgende Variablen können einen Einfluss auf die Zufriedenheit der Kunden haben 
* **aircraft**: Leider nur für 14 der 336 Datenpunkte erfasst, der Flugzeugtyp kann aber prinzipiell die Kundenzufriedenheit beeinflussen (Beispiel A380 wird von Fluggästen sehr positiv wahrgenommen) -> Flugzeugtyp könnte aber aus anderen Datenquellen ergänzt werden, wenn Flugnummer und/oder Flugsegmente bekannt sind (FlighRadar24, interne Datenbank) -> hier aber **drop**
* **route**: Auch hier nur vereinzelt Daten vorhanden, viele Passagiere bewerten ihre gesamte Reise, die oft auch aus mehreren Flugsegmenten besteht. Route könnte bei Bedarf teilweise aus **content** abgeleitet werden. Für diesen vereinfachten Fall wird das gesamte Reiseerlebnis mit Swiss betrachtet, unabhängig von Route und möglichen Umstiegen -> **drop**
* **content**: Geschriebene Bewertung der Nutzer und ist für alle Datenpunkte vorhanden. Enthält sehr viele Infos zu Route, Aircraft und allgemeinenen Zufriedenheit, die durch Natural Language Processing (NLP) extrahiert werden könnten. **Sehr wertvolle Daten, bei denen sich grundsätzlich eine weiterführende Analyse anbietet** -> werden in der Variable **writtenReview** abgespeichert 
* **type traveller**: Geschäftsreisende haben andere (oft höhere) Ansprüche an das Produkt, als Privatreisende. Allerdings sind auch hier nur unzureichende Daten vorhanden -> **drop**

In [ ]:
writtenReview = swiss['content']
swiss = swiss.drop(columns={"link", "author", "author_country", "airline_name", "title", 
                             "date", "aircraft", "route", "content", "type_traveller"})

Somit bleiben folgende Variablen übrig, die einen direkten Einfluss auf die Kundenzufriedenheit haben:
* **cabin_flown**
* **overall_rating**
* **seat_comfort_rating**
* **cabin_staff_rating**
* **food_beverages_rating**
* **inflight_entertainment_rating**
* **ground_service_rating**
* **wifi_connectivity_rating**
* **value_money_rating**
* **recommended**

### Fehlende Werte
Im nächsten Schritt wird ermittelt, wie viele fehlende Werte es pro Merkmal gibt und danach eine Strategie entwickelt diese zu ersetzen

In [ ]:
for col in swiss.columns:
    numberMissing = swiss[col].isna().sum()
    if numberMissing > 0 :
        print(f'Variable : {col} : {numberMissing} missing values')

Die Merkmale **ground_service_rating** und **wifi_connectivity_rating** fehlen bei fast allen Bewertungen. Nachfolgend wird angenommen, dass der Ground Service im Vergleich zu den direkt mit dem Flug verbundenen Kriterien wie Sitz, Flugpersonal, Essen- und Trinken, etc. eine untergeordnete Rolle spiel und vernachlässigt werden kann. Da Internet an Bord erst mit Einführung der 777-300ER flächendeckend bei Swiss eingeführt wurde (Quelle: https://www.swiss.com/corporate/DE/medien/newsroom/medienmitteilungen/media-release-20160127), wird dieses Merkmal ebenfalls vernachlässigt  

In [ ]:
# drop columns for ground_service_rating and wifi_connectivity_rating
swiss = swiss.drop(columns={"wifi_connectivity_rating", "ground_service_rating"})

In [ ]:
# show remaining columns with missing values
for col in swiss.columns:
    numberMissing = swiss[col].isna().sum()
    if numberMissing > 0 :
        print(f'Variable : {col} : {numberMissing} missing values')

In [ ]:
# get indices for rows with missing overall rating
ifeNaNIndex = swiss[swiss['inflight_entertainment_rating'].isna()==True].index

# show rows with NaN for inflight entertainment before replacement
swiss.loc[ifeNaNIndex]

In [ ]:
# Replace missing values for inflight entertainment with mean
swiss['inflight_entertainment_rating'].fillna(
    value=np.around(swiss['inflight_entertainment_rating'].mean()), inplace=True)

# show rows with replaced values
swiss.loc[ifeNaNIndex]

In [ ]:
# get indices for rows with missing overall rating
ratingNaNIndex = swiss[swiss['overall_rating'].isna()==True].index

# show rows with NaN for overall_rating
swiss.loc[ratingNaNIndex]

In [ ]:
# replace NaNs for overall rating with mean of all other ratings
for index in ratingNaNIndex:
    replaceValue = swiss[['seat_comfort_rating','cabin_staff_rating','food_beverages_rating',
                          'inflight_entertainment_rating','value_money_rating']].iloc[index].mean()
    swiss.loc[index,'overall_rating'] = max(np.around(replaceValue),1)

# show replaced values
swiss.loc[ratingNaNIndex]

In [ ]:
assert swiss.isna().sum().sum() == 0, "There are still NaN values in dataset"